In [ ]:
%load_ext autoreload
%autoreload 2

%config IPCompleter.greedy=True

In [ ]:
import numpy as np

import importlib

import snapshot_obj
import simulation_tracing
import halo_matching

In [ ]:
importlib.reload(snapshot_obj)
importlib.reload(simulation_tracing)
importlib.reload(halo_matching)

# Matching all halos between two snapshots

In [ ]:
snap1 = snapshot_obj.Snapshot("V1_LR_fix", 127)
gns1 = snap1.get_subhalos("GroupNumber")
sgns1 = snap1.get_subhalos("SubGroupNumber")
masses1 = snap1.get_subhalos('MassType')[:,1]

snap2 = snapshot_obj.Snapshot("V1_LR_fix", 126)
gns2 = snap2.get_subhalos("GroupNumber")
sgns2 = snap2.get_subhalos("SubGroupNumber")
masses2 = snap2.get_subhalos('MassType')[:,1]

In [ ]:
%time matches = halo_matching.match_snapshots(snap1,snap2)

In [ ]:
print("127 : 126")
for gn1,sgn1,m1,match in zip(gns1,sgns1,masses1,matches):
    if match < 2**32:
        gn2 = gns2[match]
        sgn2 = sgns2[match]
        m2 = masses2[match]
        print("({},{},{:.2e}) : ({},{},{:.2e})".format(gn1,sgn1,m1,gn2,sgn2,m2))
    else:
        print("({},{},{:.2e}) : ({},{})".format(gn1,sgn1,m1,-1,-1))

# Tracing all halos

In [ ]:
sim_id = "V1_LR_curvaton_p084_fix"
snap = snapshot_obj.Snapshot(sim_id, 127)

In [ ]:
snap.get_halo_number()

In [ ]:
tracer = simulation_tracing.MergerTree(snap)

In [ ]:
%time trace = tracer.trace_all(100)

In [ ]:
trace[0][:10]

In [ ]:
gns = {}; sgns = {}
for sid in range(127,99,-1):
    snap = snapshot_obj.Snapshot(sim_id, sid)
    gns[sid] = snap.get_subhalos("GroupNumber")
    sgns[sid] = snap.get_subhalos("SubGroupNumber")

In [ ]:
print(np.arange(gns[127].size)[np.logical_and(gns[127]==2, sgns[127]==0)])
print(gns[127][55], sgns[127][55])
print(gns[126][60], sgns[126][60])

In [ ]:
tr = tracer.tracer

for i,ht in enumerate(tr):
    out = "127 : ({},{})".format(gns[127][i],sgns[127][i])
    for sid in range(126,99,-1):
        idx = ht[sid]
        if idx == 2**32:
            out += ",  !"
            break
        else:
            out += ",  {} : ({},{})".format(sid, gns[sid][idx],sgns[sid][idx])
    print(out + "\n")